<h3>List all buckets<h3>

In [1]:
# Retrieve the list of existing buckets
import boto3
region= 'ap-southeast-1'
s3 = boto3.client('s3',region_name=region)
#s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  cloud-optimized-geotiff
  gcloud-recovery
  open-data-cube
  pixxel-aviris-data
  pixxel-emr-bucket
  pixxel-hyperion-data
  pixxelopendata


<h3>Download directory<h3>

In [3]:
def download_dir(prefix, local, bucket, client=boto3.client('s3')):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

In [7]:
import os
import boto3
local='Data_1'
prefix='Data'
bucket='pixxelopendata'
s3_client=boto3.client('s3')
download_dir(prefix, local, bucket, client=s3_client)

EndpointConnectionError: Could not connect to the endpoint URL: "https://pixxelopendata.s3.ap-southeast-1.amazonaws.com/Data/EO1H1430452010208110Kt/EO1H1430452010208110Kt_B079_L1GST.TIF"